<a href="https://colab.research.google.com/github/rawar/jetson-nano-obj-detect/blob/master/keras_model_to_tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Keras model converter for NVIDIAs Jetson platform

The following code snipped convertes an existing Keras model (see more at https://modelzoo.co/framework/keras) to NVIDIAs [TensorRT](https://developer.nvidia.com/tensorrt) model format.


## Import required libraries

In [0]:
import os
from google.colab import files
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mn
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
import tensorflow.contrib.tensorrt as trt

## Define local model folder and name 

In [0]:
save_pb_dir = './models/'
keras_model_filename = save_pb_dir + 'mobilenetv2-model.h5'
trt_model_filename = 'mobilenetv2-model-trt.pb'

## Save Keras model locally

In [5]:
keras_model = mn(weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
os.makedirs(save_pb_dir, exist_ok=True)

In [0]:
keras_model.save(keras_model_filename)

In [0]:
tf.keras.backend.clear_session()

## Load model to Tensorflow

In [0]:
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

In [11]:
tf.keras.backend.set_learning_phase(0) 
keras_model = load_model(keras_model_filename)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
input_names = [t.op.name for t in keras_model.inputs]
output_names = [t.op.name for t in keras_model.outputs]

In [13]:
print(input_names, output_names)

['input_1'] ['Logits/Softmax']


## Freeze model graph

In [0]:
session = tf.keras.backend.get_session()

In [15]:
frozen_graph = freeze_graph(
    session.graph, 
    session, 
    [out.op.name for out in keras_model.outputs], 
    save_pb_dir=save_pb_dir
)

Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 262 variables.
INFO:tensorflow:Converted 262 variables to const ops.


## Convert to TensorRT

In [16]:
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


In [17]:
graph_io.write_graph(trt_graph, save_pb_dir, trt_model_filename, as_text=False)

'./models/mobilenetv2-mode.pb'

## Download TensorRT model

In [19]:
!ls -lah ./models

total 42M
drwxr-xr-x 2 root root 4.0K Sep 29 09:46 .
drwxr-xr-x 1 root root 4.0K Sep 29 09:45 ..
-rw-r--r-- 1 root root  14M Sep 29 09:46 frozen_model.pb
-rw-r--r-- 1 root root  14M Sep 29 09:45 mobilenetv2-model.h5
-rw-r--r-- 1 root root  14M Sep 29 09:46 mobilenetv2-mode.pb


In [20]:
print(save_pb_dir+trt_model_filename)
files.download(save_pb_dir+trt_model_filename) 

./models/mobilenetv2-mode.pb
